In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from datetime import datetime, timedelta
np.random.seed(1)

In [0]:
from bs4 import BeautifulSoup
import requests
import time
import random
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [0]:
ks_2015_dataset = pd.read_csv('ks_2015.csv')

In [0]:
ks_2015_dataset['state'].value_counts()

failed        18491
successful    16205
Name: state, dtype: int64

In [0]:
url_ks_2015 = ks_2015_dataset['url_project']
url_ks_2015

0        https://www.kickstarter.com/projects/katyrex/s...
1        https://www.kickstarter.com/projects/205036093...
2        https://www.kickstarter.com/projects/162377676...
3        https://www.kickstarter.com/projects/149879225...
4        https://www.kickstarter.com/projects/urbandron...
                               ...                        
34691    https://www.kickstarter.com/projects/285721147...
34692    https://www.kickstarter.com/projects/lightsabe...
34693    https://www.kickstarter.com/projects/167172553...
34694    https://www.kickstarter.com/projects/163546745...
34695    https://www.kickstarter.com/projects/162803513...
Name: url_project, Length: 34696, dtype: object

In [0]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

filename_contents = 'ks_content_crawled_2015.csv'
filename_errors = 'ks_middle_errors_2015.csv'
content_list = []

for idx, url in enumerate(url_ks_2015[25000:], start=25000):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan
        
        #오류 나기 전까지 저장되어 있는 content_list에는 이어서 append하고; 그래서 content_list에 모으는 것도 유지할 수 있도록
        content_list.append([idx, contents_collected, risk_challenge])
        
        #오류 난 후부턴 다시 크롤링 할 때, 매 페이지 긁어온 것마다 바로바로 csv파일에 한줄씩 append 할 것이다
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print(idx)
    
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 4:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list.append([idx, contents_collected, risk_challenge])
                
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print(idx)
                break
            
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        #만약 재시도를 10번 했는데도 크롤링에 실패한다면, 원래 했던대로 오류 index, url, nan값을 csv파일들에 저장
        if count == 4:
            content_list.append([idx, error_url, np.nan])
            
            error_into_content_csv = []
            error_into_content_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_content_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
        
        continue

driver.close()

25000
25001
25002
25003
25004
25005
25006
25007
25008
25009
25010
25011
25012
25013
25014
25015
25016
25017
25018
25019
25020
25021
25022
25023
25024
25025
25026
25027
25028
25029
25030
25031
25032
25033
25034
25035
25036
25037
25038
25039
25040
25041
25042
25043
25044
25045
25046
25047
25048
25049
25050
25051
25052
25053
25054
25055
25056
25057
25058
25059
25060
25061
25062
25063
25064
25065
25066
25067
25068
25069
25070
25071
25072
25073
25074
25075
25076
25077
25078
25079
25080
25081
25082
25083
25084
25085
25086
25087
25088
25089
25090
25091
25092
25093
25094
25095
25096
25097
25098
25099
25100
25101
25102
25103
25104
25105
25106
25107
25108
25109
25110
25111
25112
25113
25114
25115
25116
25117
25118
25119
25120
25121
25122
25123
25124
25125
25126
25127
25128
25129
25130
25131
25132
25133
25134
25135
25136
25137
25138
25139
25140
25141
25142
25143
25144
25145
25146
25147
25148
25149
25150
25151
25152
25153
25154
25155
25156
25157
25158
25159
25160
25161
25162
25163
25164
25165
2516

26135
26136
26137
26138
26139
26140
26141
26142
26143
26144
26145
26146
26147
26148
26149
26150
26151
26152
26153
26154
26155
26156
26157
26158
26159
26160
26161
26162
26163
26164
26165
26166
26167
26168
26169
26170
26171
26172
26173
26174
26175
26176
26177
Unexpected error at 26178 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 3
Error while retrying: 'NoneType' object has no attribute 'select'
26179
26180
26181
26182
26183
26184
26185
26186
26187
26188
26189
26190
26191
26192
26193
26194
26195
26196
26197
26198
26199
26200
26201
26202
26203
26204
26205
26206
26207
26208
26209
26210
26211
26212
26213
26214
26215
26216
26217
26218
26219
26220
26221
26222
26223
26224
26225
26226
26227
26228
26229
26230
26231
26232
26233
26234
26235
26236
26237
26238


27158
27159
27160
27161
27162
27163
27164
27165
27166
27167
27168
27169
27170
27171
27172
27173
27174
27175
27176
27177
27178
27179
27180
27181
27182
27183
27184
27185
27186
27187
27188
27189
27190
27191
27192
27193
27194
27195
27196
27197
27198
27199
27200
27201
27202
27203
27204
27205
27206
27207
27208
27209
27210
27211
27212
27213
27214
27215
27216
27217
27218
27219
27220
27221
27222
27223
27224
27225
27226
27227
27228
27229
27230
27231
27232
27233
27234
27235
27236
27237
27238
27239
27240
27241
27242
27243
27244
27245
27246
27247
27248
27249
27250
27251
27252
27253
27254
27255
27256
27257
27258
27259
27260
27261
27262
27263
27264
27265
27266
27267
27268
27269
27270
27271
27272
27273
27274
27275
27276
27277
27278
27279
27280
27281
27282
27283
27284
27285
27286
27287
27288
27289
27290
27291
27292
27293
27294
27295
27296
27297
27298
27299
27300
27301
27302
27303
27304
27305
27306
27307
27308
27309
27310
27311
27312
27313
27314
27315
27316
Unexpected error at 27317 'NoneType' object ha

28289
28290
28291
28292
28293
28294
28295
28296
28297
28298
28299
28300
28301
28302
28303
28304
28305
28306
Unexpected error at 28307 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
28307
28308
28309
28310
28311
28312
28313
28314
Unexpected error at 28315 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
28315
28316
28317
28318
28319
28320
28321
Unexpected error at 28322 'NoneType' object has no attribute 'select'
Retrying: 0
28322
28323
28324
Unexpected error at 28325 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'


29337
29338
29339
29340
29341
29342
29343
29344
29345
29346
29347
29348
29349
29350
29351
29352
29353
29354
29355
29356
29357
29358
29359
29360
29361
29362
29363
29364
29365
29366
29367
29368
29369
29370
29371
29372
29373
29374
29375
29376
29377
29378
29379
29380
29381
29382
29383
29384
29385
29386
29387
29388
29389
29390
29391
29392
29393
29394
29395
29396
29397
29398
29399
29400
29401
29402
29403
29404
29405
29406
29407
29408
29409
29410
29411
29412
29413
29414
29415
29416
29417
29418
29419
29420
29421
29422
29423
29424
29425
29426
29427
29428
29429
29430
29431
29432
29433
29434
29435
29436
29437
29438
29439
29440
29441
29442
29443
29444
29445
29446
29447
Unexpected error at 29448 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
29448
29449
29450
29451
29452
29453
29454
29455
29456
29457
29458
29459
29460
29461
29462
29463
29464
29465
29466
29467
29468
29469
29470
29471
29472
29473
29474
29475
29476


30429
30430
30431
30432
30433
30434
30435
30436
30437
30438
30439
30440
30441
30442
30443
30444
30445
30446
30447
30448
30449
30450
30451
30452
30453
30454
30455
30456
30457
30458
30459
30460
30461
30462
30463
30464
30465
30466
30467
30468
30469
30470
30471
30472
30473
30474
30475
30476
30477
30478
30479
30480
30481
30482
30483
30484
30485
30486
30487
30488
30489
30490
30491
30492
30493
30494
30495
30496
30497
30498
30499
30500
30501
30502
30503
30504
30505
30506
30507
30508
30509
30510
30511
30512
30513
30514
30515
30516
30517
30518
30519
30520
30521
30522
30523
30524
30525
30526
30527
30528
30529
30530
30531
30532
30533
30534
30535
30536
30537
30538
30539
30540
30541
30542
30543
30544
30545
30546
30547
30548
30549
30550
30551
30552
30553
30554
30555
30556
30557
30558
30559
30560
30561
30562
30563
30564
30565
30566
30567
30568
30569
30570
30571
30572
30573
30574
30575
30576
30577
30578
30579
30580
30581
30582
30583
30584
30585
30586
30587
30588
30589
30590
30591
30592
30593
30594
3059

31512
31513
31514
31515
31516
31517
31518
31519
31520
31521
31522
31523
31524
31525
31526
31527
31528
31529
31530
31531
31532
31533
31534
31535
31536
31537
31538
31539
31540
31541
31542
31543
31544
31545
31546
31547
31548
31549
31550
31551
31552
31553
31554
31555
31556
31557
31558
31559
31560
31561
31562
31563
31564
31565
31566
31567
31568
Unexpected error at 31569 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 3
31569
31570
31571
31572
31573
31574
31575
31576
31577
31578
31579
31580
31581
31582
31583
31584
31585
31586
31587
31588
31589
31590
31591
31592
31593
31594
31595
31596
31597
31598
31599
31600
31601
31602
31603
31604
31605
31606
31607
31608
31609
31610
31611
31612
31613
31614
31615
31616
31617
31618
31619
31620
31621
31622
31623
31624
31625


32630
32631
32632
32633
32634
32635
32636
32637
32638
32639
32640
32641
32642
32643
32644
32645
32646
32647
32648
32649
32650
32651
32652
32653
32654
32655
32656
32657
32658
32659
32660
32661
32662
32663
32664
32665
32666
32667
32668
32669
32670
32671
32672
32673
32674
32675
32676
32677
32678
32679
32680
32681
32682
32683
32684
32685
32686
32687
32688
32689
32690
32691
32692
32693
32694
32695
32696
32697
32698
32699
32700
32701
32702
32703
32704
32705
32706
32707
32708
32709
32710
32711
32712
32713
32714
32715
32716
32717
32718
32719
32720
32721
32722
32723
32724
32725
32726
32727
32728
32729
32730
32731
32732
32733
32734
32735
32736
32737
32738
32739
32740
32741
32742
32743
32744
32745
32746
32747
32748
32749
32750
32751
32752
32753
32754
Unexpected error at 32755 'NoneType' object has no attribute 'select'
Retrying: 0
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
Error while 

Retrying: 1
Error while retrying: 'NoneType' object has no attribute 'select'
Retrying: 2
33746
33747
33748
33749
33750
33751
33752
33753
33754
33755
33756
33757
33758
33759
33760
33761
33762
33763
33764
33765
33766
33767
33768
33769
33770
33771
33772
33773
33774
33775
33776
33777
33778
33779
33780
33781
33782
33783
33784
33785
33786
33787
33788
33789
33790
33791
33792
33793
33794
33795
33796
33797
33798
33799
33800
33801
33802
33803
33804
33805
33806
33807
33808
33809
33810
33811
33812
33813
33814
33815
33816
33817
33818
33819
33820
33821
33822
33823
33824
33825
33826
33827
33828
33829
33830
33831
33832
33833
33834
33835
33836
33837
33838
33839
33840
33841
33842
33843
33844
33845
33846
33847
33848
33849
33850
33851
33852
33853
33854
33855
33856
33857
33858
33859
33860
33861
33862
33863
33864
33865
33866
33867
33868
33869
33870
33871
33872
33873
33874
33875
33876
33877
33878
33879
33880
33881
33882
33883
33884
33885
33886
33887
33888
33889
33890
33891
33892
33893
33894
33895
33896
3389

In [0]:
#2015년 content파일 불러와 합치기

In [0]:
ks_2015_1 = pd.read_csv('./data_2015/ks_content_crawled_2015_0_10000.csv')

In [0]:
ks_2015_2 = pd.read_csv('./data_2015/ks_content_crawled_2015_10000_15000.csv')

In [0]:
ks_2015_3 = pd.read_csv('./data_2015/ks_content_crawled_2015_15000_20000.csv')

In [0]:
ks_2015_4 = pd.read_csv('./data_2015/ks_content_crawled_2015_20000_25000.csv')

In [0]:
ks_2015_5 = pd.read_csv('./data_2015/ks_content_crawled_2015_25000_34695.csv')

In [0]:
ks_2015_1.head(1)

index                                            content  \
0      0  ['Meet Jane Bowles: incredible author, inspiri...   

                                      risk_challenge  
0  The main obstacles this book faces are the siz...

In [0]:
ks_2015_2.head(1)

index                                            content  \
0  10000  ['A few colleagues and I have started a small ...   

                                      risk_challenge  
0  Right now our biggest challenge is gathering a...

In [0]:
# column이 일치해야 concat을 할 수 있다
# csv파일들에 같은 컬럼 이름 추가

In [0]:
concat_content_2015 = pd.concat([ks_2015_1, ks_2015_2, ks_2015_3, ks_2015_4, ks_2015_5], ignore_index=True) #수직으로 합치기(concat)
concat_content_2015

index                                            content  \
0          0  ['Meet Jane Bowles: incredible author, inspiri...   
1          1  ["I've always been passionate about efficiency...   
2          2  ['Billet Dice', 'Billet Dice are made from bil...   
3          3  ['With this film we want to entertain you in a...   
4          4  ['The Splash Drone is a fully waterproof quad ...   
...      ...                                                ...   
34691  34691  ['Who is Captain Jamie? Watch this video to fi...   
34692  34692  ['In 2004, a pair of Master Replicas Star Wars...   
34693  34693  ['Well The Purpose of this Kickstarter is to g...   
34694  34694  ['Torrefy has left the studio after having com...   
34695  34695  ['The food desert issue has become an epidemic...   

                                          risk_challenge  
0      The main obstacles this book faces are the siz...  
1      The biggest challenge in completing this proje...  
2      The international shipping is a worry but I wa...  
3      We will be using the money to prevent any prob...  
4      We have spent countless hours and many months ...  
...                                                  ...  
34691  Potential costs increase on organic prime mate...  
34692  There is a primary safety concern around using...  
34693            ^ Read the above Risks and Challenges ^  
34694  The only challenge at this point is money, and...  
34695  Each location around the country where each el...  

[34696 rows x 3 columns]

In [0]:
concat_content_2015.info() #데이터/행 수 일치

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34696 entries, 0 to 34695
Data columns (total 3 columns):
index             34696 non-null int64
content           34669 non-null object
risk_challenge    34650 non-null object
dtypes: int64(1), object(2)
memory usage: 813.3+ KB


In [0]:
concat_content_2015.sort_values(by=['index'], inplace=True) #df.sort_values(by='col1', ascending=False, na_position='first')

In [0]:
concat_content_2015

index                                            content  \
0          0  ['Meet Jane Bowles: incredible author, inspiri...   
1          1  ["I've always been passionate about efficiency...   
2          2  ['Billet Dice', 'Billet Dice are made from bil...   
3          3  ['With this film we want to entertain you in a...   
4          4  ['The Splash Drone is a fully waterproof quad ...   
...      ...                                                ...   
34691  34691  ['Who is Captain Jamie? Watch this video to fi...   
34692  34692  ['In 2004, a pair of Master Replicas Star Wars...   
34693  34693  ['Well The Purpose of this Kickstarter is to g...   
34694  34694  ['Torrefy has left the studio after having com...   
34695  34695  ['The food desert issue has become an epidemic...   

                                          risk_challenge  
0      The main obstacles this book faces are the siz...  
1      The biggest challenge in completing this proje...  
2      The international shipping is a worry but I wa...  
3      We will be using the money to prevent any prob...  
4      We have spent countless hours and many months ...  
...                                                  ...  
34691  Potential costs increase on organic prime mate...  
34692  There is a primary safety concern around using...  
34693            ^ Read the above Risks and Challenges ^  
34694  The only challenge at this point is money, and...  
34695  Each location around the country where each el...  

[34696 rows x 3 columns]

In [0]:
concat_content_2015.isna().sum()

index              0
content           27
risk_challenge    46
dtype: int64

In [0]:
concat_content_2015['index'].duplicated().any()

False

In [0]:
if concat_content_2015.duplicated().any():
    some_duplicates = concat_content_2015[concat_content_2015.duplicated()].unique().head()
    print(f"Dataframe has one or more duplicated rows, for example:\n{some_duplicates}")

In [0]:
concat_content_2015.to_csv('./data_2015/concat_content_2015.csv', index=False, encoding='utf-8')

In [0]:
#2015년 error 파일 불러와 합치기

In [0]:
error_2015_1 = pd.read_csv('./data_2015/ks_errors_crawled_2015_0_10000.csv')

In [0]:
error_2015_2 = pd.read_csv('./data_2015/ks_errors_crawled_2015_15000_20000.csv')

In [0]:
error_2015_3 = pd.read_csv('./data_2015/ks_errors_crawled_2015_20000_25000.csv')

In [0]:
error_2015_4 = pd.read_csv('./data_2015/ks_errors_crawled_2015_25000_34695.csv')

In [0]:
concat_errors_2015 = pd.concat([error_2015_1,error_2015_2,error_2015_3,error_2015_4], ignore_index=True)
concat_errors_2015

index                                          error_url
0     209  https://www.kickstarter.com/projects/120941928...
1    1718  https://www.kickstarter.com/projects/86416286/...
2    4617  https://www.kickstarter.com/projects/201205113...
3    4810  https://www.kickstarter.com/projects/187381717...
4    8944  https://www.kickstarter.com/projects/100865083...
5    9926  https://www.kickstarter.com/projects/148441991...
6   15774  https://www.kickstarter.com/projects/573978726...
7   16322  https://www.kickstarter.com/projects/153670886...
8   18585  https://www.kickstarter.com/projects/145677351...
9   19362  https://www.kickstarter.com/projects/788076358...
10  20220  https://www.kickstarter.com/projects/fractalge...
11  20698  https://www.kickstarter.com/projects/771918627...
12  22732  https://www.kickstarter.com/projects/maqoo/a-m...
13  23885  https://www.kickstarter.com/projects/rlongwort...
14  24656  https://www.kickstarter.com/projects/pensivepr...
15  25392  https://www.kickstarter.com/projects/177372785...
16  25581  https://www.kickstarter.com/projects/566344277...
17  26178  https://www.kickstarter.com/projects/pullin/he...
18  26363  https://www.kickstarter.com/projects/385963985...
19  26754  https://www.kickstarter.com/projects/343047637...
20  26938  https://www.kickstarter.com/projects/123600845...
21  28113  https://www.kickstarter.com/projects/141763643...
22  30418  https://www.kickstarter.com/projects/179587903...
23  31190  https://www.kickstarter.com/projects/allenabbo...
24  31382  https://www.kickstarter.com/projects/27419856/...
25  32355  https://www.kickstarter.com/projects/986753290...
26  34110  https://www.kickstarter.com/projects/114309444...

In [0]:
concat_errors_2015.sort_values(by=['index'], inplace=True)

In [0]:
concat_errors_2015.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 2 columns):
index        27 non-null int64
error_url    27 non-null object
dtypes: int64(1), object(1)
memory usage: 648.0+ bytes


In [0]:
concat_errors_2015['index'].duplicated().any()

False

In [0]:
concat_errors_2015['error_url']

0     https://www.kickstarter.com/projects/120941928...
1     https://www.kickstarter.com/projects/86416286/...
2     https://www.kickstarter.com/projects/201205113...
3     https://www.kickstarter.com/projects/187381717...
4     https://www.kickstarter.com/projects/100865083...
5     https://www.kickstarter.com/projects/148441991...
6     https://www.kickstarter.com/projects/573978726...
7     https://www.kickstarter.com/projects/153670886...
8     https://www.kickstarter.com/projects/145677351...
9     https://www.kickstarter.com/projects/788076358...
10    https://www.kickstarter.com/projects/fractalge...
11    https://www.kickstarter.com/projects/771918627...
12    https://www.kickstarter.com/projects/maqoo/a-m...
13    https://www.kickstarter.com/projects/rlongwort...
14    https://www.kickstarter.com/projects/pensivepr...
15    https://www.kickstarter.com/projects/177372785...
16    https://www.kickstarter.com/projects/566344277...
17    https://www.kickstarter.com/projects/pulli

In [0]:
errors_2015_index = concat_errors_2015.set_index('index').index
errors_2015_index

Int64Index([  209,  1718,  4617,  4810,  8944,  9926, 15774, 16322, 18585,
            19362, 20220, 20698, 22732, 23885, 24656, 25392, 25581, 26178,
            26363, 26754, 26938, 28113, 30418, 31190, 31382, 32355, 34110],
           dtype='int64', name='index')

In [0]:
content_2015_errors_removed = concat_content_2015.drop(index=errors_2015_index)
content_2015_errors_removed

index                                            content  \
0          0  ['Meet Jane Bowles: incredible author, inspiri...   
1          1  ["I've always been passionate about efficiency...   
2          2  ['Billet Dice', 'Billet Dice are made from bil...   
3          3  ['With this film we want to entertain you in a...   
4          4  ['The Splash Drone is a fully waterproof quad ...   
...      ...                                                ...   
34691  34691  ['Who is Captain Jamie? Watch this video to fi...   
34692  34692  ['In 2004, a pair of Master Replicas Star Wars...   
34693  34693  ['Well The Purpose of this Kickstarter is to g...   
34694  34694  ['Torrefy has left the studio after having com...   
34695  34695  ['The food desert issue has become an epidemic...   

                                          risk_challenge  
0      The main obstacles this book faces are the siz...  
1      The biggest challenge in completing this proje...  
2      The international shipping is a worry but I wa...  
3      We will be using the money to prevent any prob...  
4      We have spent countless hours and many months ...  
...                                                  ...  
34691  Potential costs increase on organic prime mate...  
34692  There is a primary safety concern around using...  
34693            ^ Read the above Risks and Challenges ^  
34694  The only challenge at this point is money, and...  
34695  Each location around the country where each el...  

[34669 rows x 3 columns]

In [0]:
34696-27

34669

In [0]:
content_2015_errors_removed.isna().sum()

index              0
content            2
risk_challenge    19
dtype: int64

In [0]:
content_2015_errors_removed.isna().sum()

index              0
content            2
risk_challenge    19
dtype: int64

In [0]:
content_2015_errors_removed[content_2015_errors_removed['content'].isna() == True]

index content risk_challenge
12395  12401     NaN            NaN
12949  12955     NaN            NaN

In [0]:
e_recrawl = e1[e1['index'].isin([12401])]
e_recrawl

index category_name location_name country_displayable_name  usd_goal  \
12401  12401   Restaurants       Nairobi        the United States    3000.0   

       set_fundraising_period  target_goal_period_rate  date_launched_month  \
12401                 3524673                 0.000851                    4   

       launching_delay_time  disable_communication  is_starrable  \
12401                708962                  False         False   

                                             url_project       state  
12401  https://www.kickstarter.com/projects/877209136...  successful

In [0]:
two_errors = ks_2015_dataset[ks_2015_dataset.reset_index()['index'].isin([12401,12955])]
two_errors

category_name location_name country_displayable_name    usd_goal  \
12401   Restaurants       Nairobi        the United States  3000.00000   
12955        Nature         Paris                   France  2816.55865   

       set_fundraising_period  target_goal_period_rate  date_launched_month  \
12401                 3524673                 0.000851                    4   
12955                 1814400                 0.001552                    6   

       launching_delay_time  disable_communication  is_starrable  \
12401                708962                  False         False   
12955                332993                  False         False   

                                             url_project       state  
12401  https://www.kickstarter.com/projects/877209136...  successful  
12955  https://www.kickstarter.com/projects/100865083...  successful

In [0]:
two_errors.index == 12401

array([ True, False])

In [0]:
two_errors_url.values

array(['https://www.kickstarter.com/projects/877209136/blackbird-bakery?ref=discovery_category_newest',
       'https://www.kickstarter.com/projects/1008650837/reflexion-contre-nature?ref=discovery_category_newest'],
      dtype=object)

In [0]:
index 12401 - 페이지/본문 존재 --> 다시 크롤링
index 12955 - 페이지 hidden

In [0]:
12401 제거 완료

In [0]:
12401 재크롤링

In [0]:
e_recrawl.reset_index(drop=True, inplace=True)

In [0]:
e_recrawl['url_project'][0]

'https://www.kickstarter.com/projects/877209136/blackbird-bakery?ref=discovery_category_newest'

In [0]:
e_recrawl['index'][0]

12401

In [0]:
content_2015_errors_removed['index'].duplicated().any()

False

In [0]:
content_2015_errors_removed[content_2015_errors_removed['index'].isin([209,  1718,  4617,  4810,  8944,  9926, 15774, 16322, 18585,
            19362, 20220, 20698, 22732, 23885, 24656, 25392, 25581, 26178,
            26363, 26754, 26938, 28113, 30418, 31190, 31382, 32355, 34110])]

Empty DataFrame
Columns: [index, content, risk_challenge]
Index: []

In [0]:
content_2015_errors_removed.reset_index(drop=True, inplace=True)

In [0]:
content_2015_errors_removed

index                                            content  \
0          0  ['Meet Jane Bowles: incredible author, inspiri...   
1          1  ["I've always been passionate about efficiency...   
2          2  ['Billet Dice', 'Billet Dice are made from bil...   
3          3  ['With this film we want to entertain you in a...   
4          4  ['The Splash Drone is a fully waterproof quad ...   
...      ...                                                ...   
34664  34691  ['Who is Captain Jamie? Watch this video to fi...   
34665  34692  ['In 2004, a pair of Master Replicas Star Wars...   
34666  34693  ['Well The Purpose of this Kickstarter is to g...   
34667  34694  ['Torrefy has left the studio after having com...   
34668  34695  ['The food desert issue has become an epidemic...   

                                          risk_challenge  
0      The main obstacles this book faces are the siz...  
1      The biggest challenge in completing this proje...  
2      The international shipping is a worry but I wa...  
3      We will be using the money to prevent any prob...  
4      We have spent countless hours and many months ...  
...                                                  ...  
34664  Potential costs increase on organic prime mate...  
34665  There is a primary safety concern around using...  
34666            ^ Read the above Risks and Challenges ^  
34667  The only challenge at this point is money, and...  
34668  Each location around the country where each el...  

[34669 rows x 3 columns]

In [0]:
content_2015_errors_removed.to_csv('./data_2015/content_2015_errors_recrawled.csv', index=False, encoding='utf-8')

In [0]:
concat_errors_2015['index'][0]

209

In [0]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

filename_contents = './data_2015/content_2015_errors_recrawled.csv'
filename_errors = './data_2015/new_errors_recrawled_2015.csv'

for row, url in enumerate(concat_errors_2015['error_url']):
    idx = concat_errors_2015['index'][row]
    
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan
        
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print("row:{}".format(row), "index:{}".format(idx))
        
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 1:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print("Retry successful")
                print("row:{}".format(row), "index:{}".format(idx))
                break
                
            except Exception as et:
                print("Error while retrying:", et)
                #time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        if count == 1:
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
            print("Retry failed")
        continue
    
driver.close()

In [0]:
options = ChromeOptions()
options.add_argument('headless')
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36")
#path = '/Users/DaeyoungKim/Python/Final_Project/Kickstarter_2019-12-12/chromedriver' #chromedriver 위치
#driver = Chrome(executable_path=path, options=options) #컴퓨터에 Path 환경변수 설정 안 했을 때엔 여기서 이렇게 해줄 수 있음
driver = Chrome(options=options)

filename_contents = './data_2015/content_2015_errors_recrawled.csv'
filename_errors = './data_2015/new_errors_recrawled_2015.csv'

for row, url in enumerate(e_recrawl['url_project'][0:1]):
    idx = e_recrawl['index'][row]
    
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan
        
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
        print("row:{}".format(row), "index:{}".format(idx))
        
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 1:
            print("Retrying:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
                print("Retry successful")
                print("row:{}".format(row), "index:{}".format(idx))
                break
                
            except Exception as et:
                print("Error while retrying:", et)
                #time.sleep(random.uniform(100,130))
                count += 1
                continue
        
        if count == 1:
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, np.nan, np.nan])
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv(filename_contents, index=False, header=False, mode='a', encoding='UTF-8')
            
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv(filename_errors, index=False, header=False, mode='a', encoding='UTF-8')
            print("Retry failed")
        continue
    
driver.close()

row:0 index:12401


In [0]:
# 아까 content에 결측치 2개 있었는데 에러 수집에서 생략 된건지, url 찾아서 확인해보고 글 있는거면 걔네 둘 다시 크롤링

In [0]:
c_2015 = pd.read_csv('./data_2015/content_2015_errors_recrawled.csv')
c_2015

index                                            content  \
0          0  ['Meet Jane Bowles: incredible author, inspiri...   
1          1  ["I've always been passionate about efficiency...   
2          2  ['Billet Dice', 'Billet Dice are made from bil...   
3          3  ['With this film we want to entertain you in a...   
4          4  ['The Splash Drone is a fully waterproof quad ...   
...      ...                                                ...   
34691  31190  ['Hello! \xa0Thank you for your interest in my...   
34692  31382  ['In May, my school\'s film department will be...   
34693  32355  ['(Samples on the player above)', 'About this ...   
34694  34110  ['Here I plan to do what many have done before...   
34695  12401  ['As Nairobi becomes more fast-paced, quick on...   

                                          risk_challenge  
0      The main obstacles this book faces are the siz...  
1      The biggest challenge in completing this proje...  
2      The international shipping is a worry but I wa...  
3      We will be using the money to prevent any prob...  
4      We have spent countless hours and many months ...  
...                                                  ...  
34691  Some of the most difficult aspects of an Eagle...  
34692  The only risk present with this trip is its ca...  
34693  After more than a year of pre-production work,...  
34694  I currently study Italian at University and th...  
34695  1) Breaking into the market: It is true that t...  

[34696 rows x 3 columns]

In [0]:
c_2015.sort_values(by='index', inplace=True)

In [0]:
c_2015.reset_index(drop=True, inplace=True)

In [0]:
c_2015

index                                            content  \
0          0  ['Meet Jane Bowles: incredible author, inspiri...   
1          1  ["I've always been passionate about efficiency...   
2          2  ['Billet Dice', 'Billet Dice are made from bil...   
3          3  ['With this film we want to entertain you in a...   
4          4  ['The Splash Drone is a fully waterproof quad ...   
...      ...                                                ...   
34691  34691  ['Who is Captain Jamie? Watch this video to fi...   
34692  34692  ['In 2004, a pair of Master Replicas Star Wars...   
34693  34693  ['Well The Purpose of this Kickstarter is to g...   
34694  34694  ['Torrefy has left the studio after having com...   
34695  34695  ['The food desert issue has become an epidemic...   

                                          risk_challenge  
0      The main obstacles this book faces are the siz...  
1      The biggest challenge in completing this proje...  
2      The international shipping is a worry but I wa...  
3      We will be using the money to prevent any prob...  
4      We have spent countless hours and many months ...  
...                                                  ...  
34691  Potential costs increase on organic prime mate...  
34692  There is a primary safety concern around using...  
34693            ^ Read the above Risks and Challenges ^  
34694  The only challenge at this point is money, and...  
34695  Each location around the country where each el...  

[34696 rows x 3 columns]

In [0]:
c_2015.to_csv('./data_2015/content_2015_recrawled_0_34695.csv', index=False, encoding='utf-8')